In [2]:
import sys
from pathlib import Path

import torch
from torch.func import functional_call

# In a notebook, __file__ is not defined. Use the current working directory as a proxy.
REPO_ROOT = Path().resolve().parent
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from nanochat.gpt import GPT, GPTConfig

torch.manual_seed(0)



In [3]:
cfg = GPTConfig(sequence_len=32, vocab_size=65, n_layer=2, n_head=4, n_kv_head=4, n_embd=64)
model = GPT(cfg).eval()


In [10]:
idx = torch.randint(0, cfg.vocab_size, (64, 32), dtype=torch.int64)
targets = torch.randint(0, cfg.vocab_size, (64, 32), dtype=torch.int64)
idx

tensor([[56, 33, 32,  ..., 12, 60, 34],
        [51,  1,  3,  ..., 42, 10, 48],
        [25, 23, 21,  ..., 59, 47, 42],
        ...,
        [30, 45, 11,  ...,  2, 50, 34],
        [20, 18, 27,  ..., 27, 61, 60],
        [ 2, 28, 32,  ..., 24, 38, 52]])

In [9]:
import torch.nn.functional as F


In [15]:
idx.size()

torch.Size([64, 32])

In [18]:
logits.size()

torch.Size([64, 32, 65])

In [17]:
logits.view(-1,logits.size(-1)).size()

torch.Size([2048, 65])

In [19]:
targets.view(-1).size()

torch.Size([2048])

In [ ]:
logits = model(idx)
logits.size()

l = F.cross_entropy(
    logits.view(-1,logits.size(-1)),
    targets.view(-1),
    ignore_index = -1,
    reduction='mean'
)
l


tensor(4.3279, grad_fn=<NllLossBackward0>)

In [23]:
loss_pt = model(idx, targets)
loss_pt

tensor(4.3279, grad_fn=<NllLossBackward0>)

In [33]:
params = dict(model.named_parameters())
param_names = list(params.keys())
param_values = list(params.values())

In [34]:
class Wrapper(torch.nn.Module):
    def __init__(self, base, names):
        super().__init__()
        self.base = base
        self.names = names
    def forward(self, *args):
        params_dict = {n: p for n, p in zip(self.names, args[:-2])}
        return functional_call(self.base, params_dict, (args[-2], args[-1]))

In [ ]:
class Wrapper(torch.nn.Module):
    def __init__(self, base, names):
        super().__init__()
        self.base = base
        self.names = names
    def forward(self, *args):
        params_dict = {n: p for n, p in zip[tuple](self.names, args[:-2])}
        return functional_call(self.base, params_dict, (args[-2], args[-1]))

wrapper = Wrapper(model, param_names)

In [40]:
full_inputs = param_values + [idx, targets]


In [41]:
dict(model.named_parameters())

{'transformer.wte.weight': Parameter containing:
 tensor([[-1.1258, -1.1524, -0.2506,  ..., -1.2341,  1.8197, -0.5515],
         [-0.5692,  0.9200,  1.1108,  ...,  1.1648,  0.9234,  1.3873],
         [-0.8834, -0.4189, -0.8048,  ..., -0.9944, -1.1894, -1.1959],
         ...,
         [ 0.5901, -0.8325, -1.3715,  ...,  1.0564, -0.1504,  0.7420],
         [ 0.7272, -0.2612,  0.0124,  ..., -3.0357, -1.7288,  0.6020],
         [ 1.9476,  1.0077, -0.1007,  ..., -0.1173, -0.6841,  0.5988]],
        requires_grad=True),
 'transformer.h.0.attn.c_q.weight': Parameter containing:
 tensor([[-0.1147,  0.0432, -0.0319,  ...,  0.1144, -0.0859, -0.0636],
         [ 0.0551,  0.0994,  0.1193,  ..., -0.0264,  0.0643,  0.0257],
         [-0.1003,  0.0246,  0.0806,  ..., -0.0367,  0.0546,  0.0245],
         ...,
         [ 0.0677, -0.0634,  0.1034,  ...,  0.0205, -0.0545,  0.0246],
         [-0.0467,  0.0415,  0.0165,  ...,  0.0725,  0.0490,  0.0046],
         [-0.0042,  0.0554,  0.0177,  ..., -0.0344,  0

In [11]:

q = torch.tensor([[1.0, 2.0, 3.0],
                  [0.5, 1.5, -0.5],
                  [2.0, 0.0, 1.0]])
v = torch.tensor([[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6],
                  [0.7, 0.8, 0.9]])
qv = q @ v
qv
tril = torch.tril(torch.ones(3,3).to('cpu'))
att = qv.masked_fill(tril==0, float('-inf'))
att


tensor([[3.0000,   -inf,   -inf],
        [0.3000, 0.4500,   -inf],
        [0.9000, 1.2000, 1.5000]])

In [18]:

torch.exp(torch.tensor(0.3000)), torch.exp(torch.tensor(0.4500))


(tensor(1.3499), tensor(1.5683))

In [19]:
torch.exp(torch.tensor(0.3000)) + torch.exp(torch.tensor(0.4500))



tensor(2.9182)

In [20]:
torch.exp(torch.tensor(0.3000)) / (torch.exp(torch.tensor(0.3000)) + torch.exp(torch.tensor(0.4500)))


tensor(0.4626)

In [12]:
torch.softmax(att, dim=-1)

tensor([[1.0000, 0.0000, 0.0000],
        [0.4626, 0.5374, 0.0000],
        [0.2397, 0.3236, 0.4368]])

In [2]:
softmax_qv = torch.softmax(qv, dim=-1)
print("q @ v =\n", qv)
print("softmax(q @ v) =\n", softmax_qv)


NameError: name 'qv' is not defined